<a href="https://colab.research.google.com/github/lpxgis/CV_EXP/blob/master/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 旋转

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.data
import skimage.io
##旋转角度
angle = 35*np.pi/180
##
img = skimage.data.chelsea().astype(int)
# 设置新的图像大小
h,w = img.shape[0],img.shape[1]
newW = int(w*abs(np.cos(angle)) + h*abs(np.sin(angle)))+1
newH = int(w*abs(np.sin(angle)) + h*abs(np.cos(angle)))+1
####
newimg1 = np.zeros((newH,newW,3),dtype = int)
newimg2 =  np.zeros((newH,newW,3),dtype = int)
# 设置旋转矩阵
trans1 = np.array([[1,0,0],[0,-1,0],[-0.5*w,0.5*h,1]])
trans1 = trans1.dot(np.array([[np.cos(angle),-np.sin(angle),0],[np.sin(angle),np.cos(angle),0],[0,0,1]]))
trans1 = trans1.dot(np.array([[1,0,0],[0,-1,0],[0.5*newW,0.5*newH,1]]))

trans2 = np.array([[1,0,0],[0,-1,0],[-0.5*newW,0.5*newH,1]])
trans2 = trans2.dot(np.array([[np.cos(angle),np.sin(angle),0],[-np.sin(angle),np.cos(angle),0],[0,0,1]]))
trans2 = trans2.dot(np.array([[1,0,0],[0,-1,0],[0.5*w,0.5*h,1]]))
# 开始旋转
for x in range(w):
    for y in range(h):
        newPos = np.array([x,y,1]).dot(trans1)
        newimg1[int(newPos[1])][int(newPos[0])] = img[y][x]

for x in range(newW):
    for y in range(newH):
        srcPos = np.array([x,y,1]).dot(trans2)
        if srcPos[0] >= 0 and srcPos[0] < w and srcPos[1] >= 0 and srcPos[1] < h:
            #双线性内插
            bix,biy = int(srcPos[0]),int(srcPos[1]) # 取左上角坐标
            #避免最后一行溢出
            if bix < w-1 and biy < h-1:
                #X
                rgbX1 = img[biy][bix] + (img[biy][bix+1] - img[biy][bix])*(srcPos[0]-bix)
                rgbX2 = img[biy+1][bix] + (img[biy+1][bix+1] - img[biy+1][bix])*(srcPos[0]-bix)
                #Y
                rgb = rgbX1 + (rgbX2-rgbX1)*(srcPos[1]-biy)
                newimg2[y][x] = rgb
# 绘图
plt.figure(figsize=(16,12))
sub = plt.subplot(1,3,1)
sub.set_title("Src Image")
plt.imshow(img)
sub = plt.subplot(1,3,2)
sub.set_title("Src->Dest")
plt.imshow(newimg1)
sub = plt.subplot(1,3,3)
sub.set_title("Dest->Src Bilinear")
plt.imshow(newimg2)
plt.show()

# 平移

In [ ]:
#init
img = skimage.data.chelsea().astype(int)
h,w = img.shape[0],img.shape[1]
delta_x=14
delta_y=20
newW = w+delta_x
newH = h+delta_y
newimg1 = np.zeros((newH,newW,3),dtype = int)

for x in range(w):
  for y in range(h):
    newimg1[y+delta_y][x+delta_x] = img[y][x]
sub = plt.subplot(1,2,1)
sub.set_title("Src Image")
plt.imshow(img)
sub = plt.subplot(1,2,2)
sub.set_title("Src->Des")
plt.imshow(newimg1)
plt.show()

# 缩放

In [ ]:
#init
img = skimage.data.chelsea().astype(int)
h,w = img.shape[0],img.shape[1]
factor_x=0.5
factor_y=0.8
transform=np.array([[factor_x,0,0],[0,factor_y,0],[0,0,1]])
newW = w
newH = h
newimg1 = np.zeros((newH,newW,3),dtype = int)
for x in range(newW):
    for y in range(newH):
        srcPos = transform.dot(np.array([x,y,1]))
        if srcPos[0] >= 0 and srcPos[0] < w and srcPos[1] >= 0 and srcPos[1] < h:
            #双线性内插
            bix,biy = int(srcPos[0]),int(srcPos[1]) # 取左上角坐标
            #避免最后一行溢出
            if bix < w-1 and biy < h-1:
                #X
                rgbX1 = img[biy][bix] + (img[biy][bix+1] - img[biy][bix])*(srcPos[0]-bix)
                rgbX2 = img[biy+1][bix] + (img[biy+1][bix+1] - img[biy+1][bix])*(srcPos[0]-bix)
                #Y
                rgb = rgbX1 + (rgbX2-rgbX1)*(srcPos[1]-biy)
                newimg1[y][x] = rgb
sub = plt.subplot(1,2,1)
sub.set_title("Src Image")
plt.imshow(img)
sub = plt.subplot(1,2,2)
sub.set_title("Src->Des")
plt.imshow(newimg1)
plt.show()

# 镜像
基于pytorch的图像增强操作

In [ ]:
from torchvision import transforms
from PIL import Image
import torch
path='/content/CF595D13-B56B-499F-993F-DB2FA16A2A14.jpeg'#测试图片路径
img = Image.open(path)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=0)
])
new_img = transform(img)
plt.imshow(new_img)

# 错切

In [ ]:
from PIL import Image
from torchvision.transforms import RandomAffine

image = Image.open('/content/CF595D13-B56B-499F-993F-DB2FA16A2A14.jpeg')
random_affine = RandomAffine(
    degrees=0, 
    translate=(0, 0), 
    scale=None,
    shear=(10,12),  # 设置错切角度，有水平错切和垂直错切，若为 a，则仅在 x 轴错切，错切角度在 (-a, a)；
                 # 若为 (a, b)，则 a 设置 x 轴角度，b 设置 y 轴角度；
                 # 若为 (a, b, c, d)，则 x 轴角度为 (a, b)，y 轴角度为 (c, d)
    resample=False,
    fillcolor=0
)

plt.imshow(random_affine(image))
